<a href="https://colab.research.google.com/github/aniilkeles/DeepLearningNotes/blob/main/LSTM/PyTorch_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
text = """Bu ürün beklentimi fazlasıyla karşıladı.
Malzeme kalitesi oldukça yüksek.
Kullanım sırasında güven veriyor.
Kargo süreci hızlıydı ve sorunsuz teslim edildi.
Gönül rahatlığıyla tavsiye ederim ve öneririm!"""

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter # kelime frekansı hesaplamak için kullanılır.
from itertools import product # grid search için kombinasyon üretir.

In [13]:
# veri ön işleme : İlk öncelikle burda noktalama işaretlerinden kurtulacağım
# küçük harf dönüşümü yapacağım
# kelimleri böleceğim (token elde edeceğim)

words = text.replace(".","").replace("!","").lower().split()

In [14]:
words

['bu',
 'ürün',
 'beklentimi',
 'fazlasıyla',
 'karşıladı',
 'malzeme',
 'kalitesi',
 'oldukça',
 'yüksek',
 'kullanım',
 'sırasında',
 'güven',
 'veriyor',
 'kargo',
 'süreci',
 'hızlıydı',
 've',
 'sorunsuz',
 'teslim',
 'edildi',
 'gönül',
 'rahatlığıyla',
 'tavsiye',
 'ederim',
 've',
 'öneririm']

In [15]:
# kelime frekanslarını hesaplama ve indeksleme oluşturma işlemi
word_counts = Counter(words)

In [16]:
word_counts

Counter({'bu': 1,
         'ürün': 1,
         'beklentimi': 1,
         'fazlasıyla': 1,
         'karşıladı': 1,
         'malzeme': 1,
         'kalitesi': 1,
         'oldukça': 1,
         'yüksek': 1,
         'kullanım': 1,
         'sırasında': 1,
         'güven': 1,
         'veriyor': 1,
         'kargo': 1,
         'süreci': 1,
         'hızlıydı': 1,
         've': 2,
         'sorunsuz': 1,
         'teslim': 1,
         'edildi': 1,
         'gönül': 1,
         'rahatlığıyla': 1,
         'tavsiye': 1,
         'ederim': 1,
         'öneririm': 1})

In [19]:
# kelimeleri frekansa göre sıralama
vocab = sorted(word_counts,key=word_counts.get,reverse=True)

In [20]:
vocab

['ve',
 'bu',
 'ürün',
 'beklentimi',
 'fazlasıyla',
 'karşıladı',
 'malzeme',
 'kalitesi',
 'oldukça',
 'yüksek',
 'kullanım',
 'sırasında',
 'güven',
 'veriyor',
 'kargo',
 'süreci',
 'hızlıydı',
 'sorunsuz',
 'teslim',
 'edildi',
 'gönül',
 'rahatlığıyla',
 'tavsiye',
 'ederim',
 'öneririm']

In [25]:
word_to_index = {word: i for i,word in enumerate(vocab)} # key value eşleştirmesi

In [26]:
word_to_index

{'ve': 0,
 'bu': 1,
 'ürün': 2,
 'beklentimi': 3,
 'fazlasıyla': 4,
 'karşıladı': 5,
 'malzeme': 6,
 'kalitesi': 7,
 'oldukça': 8,
 'yüksek': 9,
 'kullanım': 10,
 'sırasında': 11,
 'güven': 12,
 'veriyor': 13,
 'kargo': 14,
 'süreci': 15,
 'hızlıydı': 16,
 'sorunsuz': 17,
 'teslim': 18,
 'edildi': 19,
 'gönül': 20,
 'rahatlığıyla': 21,
 'tavsiye': 22,
 'ederim': 23,
 'öneririm': 24}

In [27]:
index_to_word = {i: word for i,word in enumerate(vocab)} # indexlerden kelime eşleştirmesi

In [28]:
index_to_word

{0: 've',
 1: 'bu',
 2: 'ürün',
 3: 'beklentimi',
 4: 'fazlasıyla',
 5: 'karşıladı',
 6: 'malzeme',
 7: 'kalitesi',
 8: 'oldukça',
 9: 'yüksek',
 10: 'kullanım',
 11: 'sırasında',
 12: 'güven',
 13: 'veriyor',
 14: 'kargo',
 15: 'süreci',
 16: 'hızlıydı',
 17: 'sorunsuz',
 18: 'teslim',
 19: 'edildi',
 20: 'gönül',
 21: 'rahatlığıyla',
 22: 'tavsiye',
 23: 'ederim',
 24: 'öneririm'}

In [29]:
# eğitim verisi hazırlama
data = [(words[i],words[i+1]) for i in range(len(words)-1)]

In [30]:
data

[('bu', 'ürün'),
 ('ürün', 'beklentimi'),
 ('beklentimi', 'fazlasıyla'),
 ('fazlasıyla', 'karşıladı'),
 ('karşıladı', 'malzeme'),
 ('malzeme', 'kalitesi'),
 ('kalitesi', 'oldukça'),
 ('oldukça', 'yüksek'),
 ('yüksek', 'kullanım'),
 ('kullanım', 'sırasında'),
 ('sırasında', 'güven'),
 ('güven', 'veriyor'),
 ('veriyor', 'kargo'),
 ('kargo', 'süreci'),
 ('süreci', 'hızlıydı'),
 ('hızlıydı', 've'),
 ('ve', 'sorunsuz'),
 ('sorunsuz', 'teslim'),
 ('teslim', 'edildi'),
 ('edildi', 'gönül'),
 ('gönül', 'rahatlığıyla'),
 ('rahatlığıyla', 'tavsiye'),
 ('tavsiye', 'ederim'),
 ('ederim', 've'),
 ('ve', 'öneririm')]

In [32]:
# lstm modeli tanımlama
class LSTM(nn.Module):
  def __init__(self,vocab_size,embedding_dim,hidden_dim):
    super(LSTM,self).__init__() # bir üst sınıfın constructor'ını çağırma
    self.embedding = nn.Embedding(vocab_size,embedding_dim) # Embedding Katmanı
    self.lstm = nn.LSTM(embedding_dim,hidden_dim) # LSTM Katmanı
    self.fc = nn.Linear(hidden_dim,vocab_size )
  def forward(self,x):
    x = self.embedding(x) # inputu embedding katmanından geçiriyoruz.
    lstm_out,_ = self.lstm(x.view(1,1,-1))
    output = self.fc(lstm_out.view(1,-1))
    return output

In [33]:
model = LSTM(len(vocab),embedding_dim=8,hidden_dim=32)

In [34]:
# hyperparameter-tuning

In [35]:
# kelime listesi => tensore çevirme
def prepare_sequence(seq,to_ix):
  return torch.tensor([to_ix[w] for w in seq],dtype=torch.long)

In [36]:
# hyperparameter tuning kombinasyonlarını belirleme
embedding_sizes = [8,16]
hidden_sizes = [32,64]
learning_rates = [0.01,0.005]

best_loss = float("inf") # en küçük kayıp değerini saklamak için bir değişken
best_params = {}

In [43]:
for emb_size, hidden_size, lr in product(embedding_sizes, hidden_sizes, learning_rates):  # kombinasyonları elde etme
    print(f"Deneme : Embedding : {emb_size} , Hidden : {hidden_size}, Learning Rate : {lr}")

    model = LSTM(len(vocab), emb_size, hidden_size)  # seçilen parametreler ile model oluşumu
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    epochs = 50
    total_loss = 0

    for epoch in range(epochs):
        epoch_loss = 0

        for word, next_word in data:
            model.zero_grad()
            input_tensor = prepare_sequence([word], word_to_index)  # inputu tensöre çevirme işlemi
            target_tensor = prepare_sequence([next_word], word_to_index)  # target kelimeyi tensöre çevirme

            output = model(input_tensor)  # prediction
            loss = loss_function(output, target_tensor)

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        if epoch % 10 == 0:
            print(f"Epoch : {epoch}, Loss : {epoch_loss:.5f}")

        total_loss = epoch_loss

    # en iyi modeli kaydedeceğim.
    if total_loss < best_loss:
        best_loss = total_loss
        best_params = {
            "embedding_dim": emb_size,
            "hidden_dim": hidden_size,
            "learning_rate": lr
        }

    print()

print(f"Best params : {best_params}")


Deneme : Embedding : 8 , Hidden : 32, Learning Rate : 0.01
Epoch : 0, Loss : 82.33707
Epoch : 10, Loss : 3.52887
Epoch : 20, Loss : 2.17066
Epoch : 30, Loss : 1.92647
Epoch : 40, Loss : 1.86717

Deneme : Embedding : 8 , Hidden : 32, Learning Rate : 0.005
Epoch : 0, Loss : 81.41839
Epoch : 10, Loss : 13.00366
Epoch : 20, Loss : 3.35833
Epoch : 30, Loss : 2.35746
Epoch : 40, Loss : 2.03525

Deneme : Embedding : 8 , Hidden : 64, Learning Rate : 0.01
Epoch : 0, Loss : 82.29725
Epoch : 10, Loss : 2.61604
Epoch : 20, Loss : 2.03180
Epoch : 30, Loss : 1.88654
Epoch : 40, Loss : 1.79297

Deneme : Embedding : 8 , Hidden : 64, Learning Rate : 0.005
Epoch : 0, Loss : 81.81228
Epoch : 10, Loss : 5.15540
Epoch : 20, Loss : 2.30771
Epoch : 30, Loss : 1.95601
Epoch : 40, Loss : 1.83639

Deneme : Embedding : 16 , Hidden : 32, Learning Rate : 0.01
Epoch : 0, Loss : 82.41100
Epoch : 10, Loss : 3.12570
Epoch : 20, Loss : 2.12710
Epoch : 30, Loss : 1.92347
Epoch : 40, Loss : 1.86010

Deneme : Embedding : 

In [44]:
# lstm training
final_model = LSTM(len(vocab),best_params['embedding_dim'],best_params['hidden_dim'])
optimizer = optim.Adam(final_model.parameters(),lr = best_params['learning_rate'])
loss_function = nn.CrossEntropyLoss()
epochs = 100
for epoch in range(epochs):
  epoch_loss = 0
  for word,next_word in data:
    final_model.zero_grad()
    input_tensor = prepare_sequence([word],word_to_index)
    target_tensor = prepare_sequence([next_word],word_to_index)
    output = final_model(input_tensor)
    loss = loss_function(output,target_tensor)
    loss.backward()
    optimizer.step()
    epoch_loss += loss.item()
  if epoch % 10 == 0:
    print(f"Final Model Epoch : {epoch} , Loss : {epoch_loss:.5f}")

Final Model Epoch : 0 , Loss : 82.10158
Final Model Epoch : 10 , Loss : 2.35492
Final Model Epoch : 20 , Loss : 1.94091
Final Model Epoch : 30 , Loss : 1.83020
Final Model Epoch : 40 , Loss : 1.76123
Final Model Epoch : 50 , Loss : 1.71755
Final Model Epoch : 60 , Loss : 1.69197
Final Model Epoch : 70 , Loss : 1.64807
Final Model Epoch : 80 , Loss : 1.63767
Final Model Epoch : 90 , Loss : 1.62642


In [53]:
# test ve değerlendirme
# kelime tahmini fonksiyonu : baslangic kelimesi ve n adet kelime üretimi
def predict_sequence(start_word,num_words):
  current_word = start_word
  output_sequence = [current_word]

  for _ in range(num_words):
    with torch.no_grad():
      input_tensor = prepare_sequence([current_word],word_to_index)
      output = final_model(input_tensor)
      predicted_idx = torch.argmax(output).item() # en yüksek olasılığa sahip kelimenin indeksi
      predicted_word = index_to_word[predicted_idx] # indekse karşılık gelen kelimeyi  return edecek
      output_sequence.append(predicted_word)
      current_word = predicted_word

  return output_sequence

In [58]:
"""Bu ürün beklentimi fazlasıyla karşıladı.
Malzeme kalitesi oldukça yüksek.
Kullanım sırasında güven veriyor.
Kargo süreci hızlıydı ve sorunsuz teslim edildi.
Gönül rahatlığıyla tavsiye ederim ve öneririm!"""

'Bu ürün beklentimi fazlasıyla karşıladı.\nMalzeme kalitesi oldukça yüksek.\nKullanım sırasında güven veriyor.\nKargo süreci hızlıydı ve sorunsuz teslim edildi.\nGönül rahatlığıyla tavsiye ederim ve öneririm!'

In [57]:
start_word = "ve"
num_predictions = 2
predicted_sequence = predict_sequence(start_word,num_predictions)
print(" ".join(predicted_sequence))

ve sorunsuz teslim
